In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import HistGradientBoostingRegressor
import numpy as np

In [2]:
def load_datasets():
    nasdaq_data = pd.read_csv("datasets/NASDAQ_100.csv")
    QQQ_data = pd.read_csv("datasets/QQQ_raw.csv")
    QQQ_split_data = pd.read_csv("datasets/QQQ_split_adj.csv")
    SPY500_data = pd.read_csv("datasets/SP500.csv")
    SPY_data = pd.read_csv("datasets/SPY.csv")

Predicting close

In [3]:
data = pd.read_csv('datasets/SPY.csv')

# Prepare the data
X = data[['open', 'high', 'low', 'volume', 'change_percent', ]]  # Features
y = data['close']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a Random Forest Regressor
hgb_regressor = HistGradientBoostingRegressor(random_state=42)

# Train the model
hgb_regressor.fit(X_train, y_train)

# Make predictions on the test set
y_pred = hgb_regressor.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

# Make predictions for future stock prices
future_data = pd.DataFrame({
    'open': [400.0],
    'high': [410.0],
    'low': [395.0],
    'volume': [1000000],
    'change_percent': [0.5]
})

future_pred = hgb_regressor.predict(future_data)

print("Predicted future stock price:", future_pred)

Mean Squared Error: 1.8139333254779888
R-squared: 0.999855479349169
Predicted future stock price: [403.11350685]


Predicting next day

In [16]:
df = pd.read_csv('datasets/SPY.csv')

# Assuming the target column is at index 4 (5th column)
target_column = 4

# Shift the target column by -1 to get the next row's value
df['next_day_target'] = df.iloc[:, target_column].shift(-1)

# Create new columns for previous day's features
prev_day_columns = ['open', 'high', 'low', 'close', 'volume', 'change_percent']
for column in prev_day_columns:
    df[f'prev_day_{column}'] = df[column].shift(1)

# Drop the last row since it will have missing values for the next day's target and previous day's features
df = df[:-1]


prev_week_columns = ['open', 'high', 'low', 'close', 'volume', 'change_percent']
for column in prev_week_columns:
    df[f'prev_week_{column}'] = df[column].shift(5)
# Save the modified DataFrame back to a CSV file
df.to_csv('modified_file.csv', index=False)

data = pd.read_csv('modified_file.csv')

X = data[['open', 'high', 'low', 'close', 'volume', 'change_percent']]  # Features
y = data['next_day_target']  # Target variable


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


hgb_regressor = HistGradientBoostingRegressor(random_state=42)

# Train the model
hgb_regressor.fit(X_train, y_train)

# Make predictions on the test set
y_pred = hgb_regressor.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

future_data = pd.DataFrame({
    'open': [400.0],
    'high': [410.0],
    'low': [395.0],
    'close': [408.0],
    'volume': [1000000],
    'change_percent': [0.5]
})

future_pred = hgb_regressor.predict(future_data)

print("Predicted next day stock price:", future_pred)

Mean Squared Error: 7.447234231827849
R-squared: 0.9993909052533178
Predicted next day stock price: [407.8130068]
